In [1]:
import sys
import time
sys.path.append("..")
from Functions.UNSW_DF import *

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from keras_visualizer import visualizer
from ann_visualizer.visualize import ann_viz
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

x_train, x_test, y_train, y_test = DF_XY()

Using TensorFlow backend.


( 1 ) Reading Preprocessed CSV files..
	 Training dataset loaded..
	 Testing dataset loaded..

( 2 ) Loading done, splitting into X and Y..
	 ( 2.1 ) x_train Shape:  	 (175341, 53)
	 ( 2.2 ) y_train Shape:  	 (175341,)
	 ( 2.3 ) x_test Shape:  	 (82332, 53)
	 ( 2.4 ) y_test Shape:  	 (82332,)
( 3 ) Done!
PS! Import with: x_train, x_test, y_train, y_test = XY_import()


In [2]:
# ACCURACY
def get_accuracy_error(dataset, yhat_classes, train_test_string):
    """Get accuracy and error rate from given keras model.

    Args:
        dataset (dataframe): Dataframe for the model, either y_train or y_test
        yhat_classes (prediction): model prediction for crisp classes
        train_test_string (string): Represent string either as "train" or "test" for correct output

    Returns:
        float: returns accuracy and error rate for given model and datasets.
    """
    accuracy = accuracy_score(dataset, yhat_classes)
    
    if train_test_string == "train":
        # Train accuracy
        train_accuracy = accuracy
        print(f'Accuracy for training:\t {train_accuracy}')
        # Error rate for train
        train_error = 1 - train_accuracy
        print('Error for training:\t %f' %train_error)
        return train_accuracy, train_error
    elif train_test_string == "test":
        # Test accuracy
        test_accuracy = accuracy
        print(f'Accuracy for training:\t {test_accuracy}')
        # Error rate for test
        test_error = 1 - test_accuracy
        print('Error for testing:\t %f' %test_error)
        return test_accuracy, test_error
    else:
        print("Type either 'train' or 'test' for train_test_string parameter")

# PRECISION
def get_precision(dataset, yhat_classes, train_test_string):
    """Get precision for given keras model: Precision = tp / (tp + fp)
    Args:
        dataset (dataframe): Dataframe for the model, either y_train or y_test
        yhat_classes (prediction): model prediction for crisp classes
        train_test_string (string): Represent string either as "train" or "test" for correct output

    Returns:
        float:  Returns precision for given model and dataset
    """
    precision = precision_score(dataset, yhat_classes)
    
    if train_test_string == "train":
        train_precision = precision
        print('Precision for training:\t %f' % train_precision)
        return train_precision
    elif train_test_string == "test":
        test_precision = precision
        print('Precision for testing:\t %f' % test_precision)
        return test_precision
    else:
        print("Type either 'train' or 'test' for train_test_string parameter")

# RECALL
def get_recall(dataset, yhat_classes, train_test_string):
    """Get Recall for given keras model: Recall = tp / (tp + fn)
    Args:
        dataset (dataframe): Dataframe for the model, either y_train or y_test
        yhat_classes (prediction): model prediction for crisp classes
        train_test_string (string): Represent string either as "train" or "test" for correct output

    Returns:
        float:  Returns recall for given model and dataset
    """
    recall = recall_score(dataset, yhat_classes)
    
    if train_test_string == "train":
        train_recall = recall
        print('Recall for training:\t %f' % train_recall)
        return train_recall
    elif train_test_string == "test":
        test_recall = recall
        print('Recall for testing:\t %f' % test_recall)
        return test_recall
    else:
        print("Type either 'train' or 'test' for train_test_string parameter")
        
# F1
def get_F1(dataset, yhat_classes, train_test_string):
    """Get F1 for given keras model: F1 = 2 tp / (2 tp + fp + fn)
    Args:
        dataset (dataframe): Dataframe for the model, either y_train or y_test
        yhat_classes (prediction): model prediction for crisp classes
        train_test_string (string): Represent string either as "train" or "test" for correct output

    Returns:
        float:  Returns F1 for given model and dataset
    """
    f1 = f1_score(dataset, yhat_classes)
    
    if train_test_string == "train":
        train_f1 = f1
        print('F1 for training:\t\t %f' % train_f1)
        return train_f1
    elif train_test_string == "test":
        test_f1 = f1
        print('F1 for testing:\t\t %f' % test_f1)
        return test_f1
    else:
        print("Type either 'train' or 'test' for train_test_string parameter")

In [3]:
# Empty lists' for accuracy
ann_train_accuracy, ann_test_accuracy = [], []
# Empty lists' for precision
ann_train_precision, ann_test_precision = [], []
# Empty lists' for accuracy
ann_train_recall, ann_test_recall= [], []
# Empty lists' for F1
ann_train_f1, ann_test_f1= [], []
# Empty lists' for error rate
ann_train_error, ann_test_error = [], []
# Empty lists' for attributes
ann_epochs, ann_no_of_layers, ann_runtime= [], [], []

def ANN_predict(epoch_start, epoch_end, epoch_step, batch_size):
    """Predicts an ANN model with a predefined model.

    Args:
        epoch_start (int): Start value for Epoch
        epoch_end (int): End value for Epoch
        bactch_size (int): Keras batch size
    """
    for epoch in range(epoch_start, epoch_end + 1, epoch_step):
        # define the keras model
        model = Sequential()
        model.add(Dense(500, input_dim=53, activation='relu'))
        model.add(Dense(250, activation='relu'))
        model.add(Dense(100, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))

        # compile the keras model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        predict_x = model.predict(x_test) 
        classes_x = np.argmax(predict_x,axis=1)
        
        # Start time for calculating the runtime for each epoch
        start_time = time.time()
        print(f"\n## ---------- EPOCH {epoch} ----------- ##\n")
        # fit the keras model on the dataset
        model.fit(x_train, y_train, epochs=epoch, batch_size=batch_size)
        
        ###------------------------###
        ###--- MODEL PREDICTION ---###
        ###------------------------###
        # predict probabilities for train set
        yhat_probs_train = model.predict(x_train, verbose=0)
        # predict crisp classes for train set
        yhat_classes_train = model.predict_classes(x_train, verbose=0)
        
        # predict probabilities for test set
        yhat_probs_test = model.predict(x_test, verbose=0)
        # predict crisp classes for test set
        yhat_classes_test = model.predict_classes(x_test, verbose=0)
        
        ###--------------------###
        ###--- MODEL SCORES ---###
        ###--------------------###
        ###--- ACCURACY & ERROR ---###
        train_accuracy, train_error = get_accuracy_error(y_train, yhat_classes_train, "train")
        test_accuracy, test_error = get_accuracy_error(y_test, yhat_classes_test, "test")

        ###--- PRECISION ---###
        train_precision = get_precision(y_train, yhat_classes_train, "train")
        test_precision = get_precision(y_test, yhat_classes_test, "test")
        
        ###--- RECALL ---###
        train_recall = get_recall(y_train, yhat_classes_train, "train")
        test_recall = get_recall(y_test, yhat_classes_test, "test")
        
        ###--- F1 ---###
        train_f1 = get_recall(y_train, yhat_classes_train, "train")
        test_f1 = get_recall(y_test, yhat_classes_test, "test")

        # Appending Scores to lists'
        ann_train_accuracy.append(train_accuracy)
        ann_test_accuracy.append(test_accuracy)
        
        ann_train_error.append(train_error)
        ann_test_error.append(test_error)
        
        ann_train_precision.append(train_precision)
        ann_test_precision.append(test_precision)
        
        ann_train_recall.append(train_recall)
        ann_test_recall.append(test_recall)
        
        ann_train_f1.append(train_f1)
        ann_test_f1.append(test_f1)
        
        # Appending attributes
        ann_no_of_layers.append(len(model.layers))
        ann_epochs.append(epoch)
        elapsed_time = round((time.time() - start_time), 3)
        ann_runtime.append(elapsed_time)
        print(f"Runtime for Epoch {epoch}:\t {elapsed_time}s")

In [4]:
ANN_predict(epoch_start=0, epoch_end=200, epoch_step=10, batch_size=100)

2022-02-05 08:47:17.709193: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-05 08:47:17.709560: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 16. Tune using inter_op_parallelism_threads for best performance.



## ---------- EPOCH 0 ----------- ##

Accuracy for training:	 0.5293000496176022
Error for training:	 0.470700
Accuracy for training:	 0.5293446047709275
Error for testing:	 0.470655
Precision for training:	 0.789380
Precision for testing:	 0.597062
Recall for training:	 0.420669
Recall for testing:	 0.446572
Recall for training:	 0.420669
Recall for testing:	 0.446572
Runtime for Epoch 0:	 27.646s

## ---------- EPOCH 10 ----------- ##

Epoch 1/10
175341/175341 [==============================] - 21s 119us/step - loss: 0.1315 - accuracy: 0.9364
Epoch 2/10
175341/175341 [==============================] - 20s 114us/step - loss: 0.1200 - accuracy: 0.9406
Epoch 3/10
175341/175341 [==============================] - 20s 115us/step - loss: 0.1174 - accuracy: 0.9420
Epoch 4/10
175341/175341 [==============================] - 20s 114us/step - loss: 0.1153 - accuracy: 0.9435
Epoch 5/10
175341/175341 [==============================] - 20s 115us/step - loss: 0.1139 - accuracy: 0.9438
Epoch 6/10
1

# Neural Network

In [ ]:
# dictionary of lists 
dict = {'epochs': ann_epochs, 
        'no_layers': ann_no_of_layers, 
        'accuracy_train': ann_train_accuracy, 
        'accuracy_test': ann_test_accuracy, 
        'error_train': ann_train_error, 
        'error_test': ann_test_error, 
        'precision_train': ann_train_precision, 
        'precision_test': ann_test_precision, 
        'F1_train': ann_train_f1,
        'F1_test': ann_test_f1,  
        'recall_train': ann_train_recall,
        'recall_test': ann_test_recall,
        'runtime(s)': ann_runtime
        } 
    
df = pd.DataFrame(dict)
# EXPORT AS CSV when done.
df.to_csv('ANN_200E_4L(500-250-100-1)_AF(Relu-Sigmoid).csv', index=False)
df

In [ ]:
#UNSW_barplot(data=df, to_range=29 ,x_label="epochs", y_label="Score", title="ANN", x_size=30, y_size=8)

In [ ]:
#ann_viz(model, title="Neural Network")

In [ ]:
#visualizer(model, format='png', view=True)

In [ ]:
# evaluate the keras model for training and testing data
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy for training\t: %.2f' % (train_acc * 100))
print('Accuracy for testing\t: %.2f' % (test_acc * 100))